In [1]:
import os
import bs4
import bz2
import tqdm
import wget
import requests
import subprocess
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [2]:
base_url = 'https://dumps.wikimedia.org/enwiki/'
index = requests.get(base_url).text
soup_index = bs4.BeautifulSoup(index, 'html.parser')

dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]
dumps

['../',
 '20200220/',
 '20200301/',
 '20200401/',
 '20200420/',
 '20200501/',
 '20200520/',
 '20200601/',
 'latest/']

In [3]:
dump_url = base_url + '20200520/'

# Retrieve the html
dump_html = requests.get(dump_url).text
print(dump_html[:500])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"
        "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
        <title>enwiki dump progress on 20200520</title>
        <link rel="stylesheet" type="text/css" href="/dumps.css" />
        <style type="text/css">
                .siteinfo {
                        text-align: center;
 


In [4]:
soup_dump = bs4.BeautifulSoup(dump_html, 'html.parser')

files = []
for file in soup_dump.find_all('li', {'class': 'file'}):
    text = file.text
    if 'pages-articles' in text:
        files.append((text.split()[0], text.split()[1:]))
        
files[:5]

[('enwiki-20200520-pages-articles-multistream.xml.bz2', ['17.0', 'GB']),
 ('enwiki-20200520-pages-articles-multistream-index.txt.bz2', ['211.8', 'MB']),
 ('enwiki-20200520-pages-articles-multistream1.xml-p1p30303.bz2',
  ['178.1', 'MB']),
 ('enwiki-20200520-pages-articles-multistream-index1.txt-p1p30303.bz2',
  ['163', 'KB']),
 ('enwiki-20200520-pages-articles-multistream2.xml-p30304p88444.bz2',
  ['211.9', 'MB'])]

In [5]:
files_to_download = [file[0] for file in files if '.xml-p' in file[0]]
len(files_to_download), files_to_download[-5:]

(118,
 ['enwiki-20200520-pages-articles27.xml-p57663462p59163461.bz2',
  'enwiki-20200520-pages-articles27.xml-p59163462p60663461.bz2',
  'enwiki-20200520-pages-articles27.xml-p60663462p62163461.bz2',
  'enwiki-20200520-pages-articles27.xml-p62163462p63663461.bz2',
  'enwiki-20200520-pages-articles27.xml-p63663462p64021337.bz2'])

In [10]:
%%time
if not os.path.exists('./wikipedia'):
    os.makedirs('./wikipedia')

data_paths, file_info = [], []
for i, file in enumerate(files_to_download):
    clear_output(wait=True)
    print(f'File {i + 1} / {len(files_to_download)} ...\n')
    path = os.path.join('./wikipedia', file)
    if not os.path.exists(path):
        _ = wget.download(os.path.join(dump_url, file), out=path)
        data_paths.append(path)
        file_size = os.stat(path).st_size / 1e6
        file_articles = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file, file_size, file_articles))
    else:
        data_paths.append(path)
        file_size = os.stat(path).st_size / 1e6
        file_number = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file.split('-')[-1], file_size, file_number))
    print('\nOK')

File 118 / 118 ...

100% [.......................................................] 80690052 / 80690052
OK
Wall time: 2h 14min 36s
